In [3]:
import org.apache.spark.sql.SQLContext
//val sqlContext = new SQLContext(sc);
import org.apache.spark.sql.types._
//https://docs.microsoft.com/en-us/azure/databricks/_static/notebooks/movie-recommendation-engine.html

import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types._

In [4]:
val moviesdf = spark.read.format("csv").option("header", "false").option("inferSchema", "true").option("delimiter", ":").load("abfss://opendataset@internalsandboxwe.dfs.core.windows.net/movies/movies.dat")

moviesdf: org.apache.spark.sql.DataFrame = [_c0: int, _c1: string ... 3 more fields]

In [35]:
display(moviesdf)

In [5]:
val moviesdf1 = moviesdf.select("_c0","_c2","_c4").withColumnRenamed("_c0","MovieID").withColumnRenamed("_c2","MovieName").withColumnRenamed("_c4","Genre")

moviesdf1: org.apache.spark.sql.DataFrame = [MovieID: int, MovieName: string ... 1 more field]

In [6]:
display(moviesdf1)

In [7]:
val ratingsdf = spark.read.format("csv").option("header", "false").option("delimiter", ":").load("abfss://opendataset@internalsandboxwe.dfs.core.windows.net/movies/ratings.dat")

ratingsdf: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 5 more fields]

In [8]:
val ratingsdf1 = ratingsdf.select("_c0","_c2","_c4","_c6").withColumnRenamed("_c0","UserID").withColumnRenamed("_c2","MovieID").withColumnRenamed("_c4","Rating").withColumnRenamed("_c6","Timestamp")

ratingsdf1: org.apache.spark.sql.DataFrame = [UserID: string, MovieID: string ... 2 more fields]

In [9]:
display(ratingsdf1)

In [10]:
val usersdf = spark.read.format("csv").option("header", "false").option("delimiter", ":").load("abfss://opendataset@internalsandboxwe.dfs.core.windows.net/movies/users.dat")

usersdf: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 7 more fields]

In [11]:
val usersdf1 = usersdf.select("_c0","_c2","_c4","_c6","_c8").withColumnRenamed("_c0","UserID").withColumnRenamed("_c2","Gender").withColumnRenamed("_c4","Age").withColumnRenamed("_c6","Occupation").withColumnRenamed("_c8","ZipCode")

usersdf1: org.apache.spark.sql.DataFrame = [UserID: string, Gender: string ... 3 more fields]

In [12]:
display(usersdf1)

In [13]:
moviesdf1.count
ratingsdf1.count
usersdf1.count

res8: Long = 3883
res9: Long = 1000209
res10: Long = 6040

In [14]:
moviesdf1.schema
ratingsdf1.schema
usersdf1.schema

res11: org.apache.spark.sql.types.StructType = StructType(StructField(MovieID,IntegerType,true), StructField(MovieName,StringType,true), StructField(Genre,StringType,true))
res12: org.apache.spark.sql.types.StructType = StructType(StructField(UserID,StringType,true), StructField(MovieID,StringType,true), StructField(Rating,StringType,true), StructField(Timestamp,StringType,true))
res13: org.apache.spark.sql.types.StructType = StructType(StructField(UserID,StringType,true), StructField(Gender,StringType,true), StructField(Age,StringType,true), StructField(Occupation,StringType,true), StructField(ZipCode,StringType,true))

In [62]:
display(moviesdf1)

In [15]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS

import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS

In [17]:
ratingsdf1.schema

res15: org.apache.spark.sql.types.StructType = StructType(StructField(UserID,StringType,true), StructField(MovieID,StringType,true), StructField(Rating,StringType,true), StructField(Timestamp,StringType,true))

In [18]:
val df2 = ratingsdf1.selectExpr("cast(UserID as int) UserID", "cast(MovieID as int) MovieID", "cast(Rating as int) Rating")

df2: org.apache.spark.sql.DataFrame = [UserID: int, MovieID: int ... 1 more field]

In [20]:
val Array(training, test) = df2.randomSplit(Array(0.8, 0.2))

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [UserID: int, MovieID: int ... 1 more field]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [UserID: int, MovieID: int ... 1 more field]

In [21]:
val als = new ALS().setMaxIter(5).setRegParam(0.01).setUserCol("UserID").setItemCol("MovieID").setRatingCol("Rating")
val model = als.fit(training)

als: org.apache.spark.ml.recommendation.ALS = als_389ee82e4f1e
model: org.apache.spark.ml.recommendation.ALSModel = als_389ee82e4f1e

In [22]:
// Evaluate the model by computing the RMSE on the test data
// Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
model.setColdStartStrategy("drop")
val predictions = model.transform(test)

res18: model.type = als_389ee82e4f1e
predictions: org.apache.spark.sql.DataFrame = [UserID: int, MovieID: int ... 2 more fields]

In [24]:
val evaluator = new RegressionEvaluator().setMetricName("rmse").setLabelCol("Rating").setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")

evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_7d7abf998ab0
rmse: Double = 0.8941636749318016
Root-mean-square error = 0.8941636749318016

In [25]:
// Generate top 10 movie recommendations for each user
val userRecs = model.recommendForAllUsers(10)
// Generate top 10 user recommendations for each movie
val movieRecs = model.recommendForAllItems(10)

userRecs: org.apache.spark.sql.DataFrame = [UserID: int, recommendations: array<struct<MovieID:int,rating:float>>]
movieRecs: org.apache.spark.sql.DataFrame = [MovieID: int, recommendations: array<struct<UserID:int,rating:float>>]